collect facebook profiles?



In [ ]:
var {JSDOM} = require('jsdom');
var fs = require('fs');
var path = require('path');
var glob = require('glob');
var assert = require('assert');
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');
var loginFacebook,
    likeAllPosts,
    getAllXPath,
    scrapeFacebookFriends;

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

function parseFacebookFriends() {
    return getAllXPath([
        '//a[contains(@href, "friends_tab")]/@href'
    ])
    .then(friends => {
        return friends
          .filter((elem, pos, arr) => arr.indexOf(elem) === pos)
          .map(f => f.replace(/\?.*$/ig, ''))
    })
}

var FRIENDS_FILE;
function collectFacebookProfiles() {
    var fresh;
    return runSeleniumCell([
        'log in facebook',
        'like all facebook posts',
        'scrape facebook profile',
        'scrape facebook friends',
    ])
        .then(r => ({
            loginFacebook,
            likeAllPosts,
            scrapeFacebookFriends
        } = r).loginFacebook())
        // TODO: abstract this data collection from JSDOM
        // TODO: only scrape once per day? use last file instead?
        //.then(() => scrapeFacebookFriends())
        .then(() => {
            var friends = glob.sync('**/*friend*', {cwd: project, nodir: true});
            friends.sort((a, b) => 
                new Date(fs.statSync(path.join(project, a)).mtime).getTime()
                    - new Date(fs.statSync(path.join(project, b)).mtime).getTime());
            FRIENDS_FILE = path.join(project, friends.pop());
            return fs.readFileSync(FRIENDS_FILE).toString();
        })
        .then(doc => {
            // call script to get all Facebook friends
            var dom = new JSDOM(doc);
            getAllXPath = importer.import('all elements xpath',  {
                client: {
                    execute: (func, ...args) => Promise.resolve({
                        value: func.apply(dom.window.document, args)
                    }),
                    addCommand: () => {
                    }
                },
                document: dom.window.document,
                XPathResult: {ORDERED_NODE_ITERATOR_TYPE: 5}
            })
            return parseFacebookFriends();
        })
        .then(friends => {
            assert(friends.length > 0, FRIENDS_FILE + ' should have friends links in it')
            // use glob.sync to make sure every friend is hit at least once in a rotation
            var existingPosts = glob.sync('**/*-posts-*.json', {cwd: project});
            // TODO: way to tell which part of the URL is unique?
            var friendCount = friends.length;
            fresh = friends.filter(profile => {
                const profileId = profile.replace(/^\/|\/$/ig, '').split('/').pop()
                .replace(/[^a-z0-9]/ig, '_');
                // TODO: check for file.stat instead of year?
                return existingPosts.indexOf(profileId
                    + '-posts-' + (new Date()).getFullYear() + '.json') === -1;
            });
            const percent = Math.round((friendCount - fresh.length) / friendCount * 100);
            console.log((friendCount - fresh.length) + ' / ' + friendCount + ' : '
                + percent
                + '%');
            //return percent === 100 ? scrapeFacebookFriends() : [];
        })
        .then(() => {
            const rand = Math.round(fresh.length * Math.random());
            console.log(fresh.slice(rand, rand + 1));
            return importer.runAllPromises(fresh.slice(rand, rand + 1)
                .map(p => (resolve) => likeAllPosts(p, null).then(r => resolve(r))));
        })
};
module.exports = collectFacebookProfiles;


In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

$$.async();
var loginFacebook, likeAllPosts;
runSeleniumCell([
    'log in facebook',
    'like all facebook posts',
])
    .then(r => {
        ({
            loginFacebook,
            likeAllPosts,
        } = r);
        return likeAllPosts('https://www.facebook.com/david.earl.smith?composeropen=1', true);
    })
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


In [ ]:
var importer = require('../Core');
var collectFacebookProfiles = importer.import('collect facebook profiles')
/*

    https://www.facebook.com/dadsrawesome/videos/1165913990203850/
    https://www.facebook.com/galacticempireofficial/videos/800461353413445/
        */
$$.async();
collectFacebookProfiles()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


sync facebook contacts with google contacts?



connect on facebook?


In [ ]:
function friendsOnFacebook(profile, friends) {
    return client
        .getUrl()
        .then(url => url.indexOf(profile) > -1 ? [] : client.url('https://' + profile))
        .isExisting('//*[@id = "fbProfileCover"]//button[contains(., "Add Friend")]')
        .then(is => is ? client
              .isVisible('//*[@id = "fbProfileCover"]//button[contains(., "Add Friend")]')
              .then(is => is
                    ? client.click('//*[@id = "fbProfileCover"]//button[contains(., "Add Friend")]')
                    : [])
              : [])
        .pause(1000)
        .isExisting('//button[contains(., "Confirm")]')
        .then(is => is ? client
              .isVisible('//button[contains(., "Confirm")]')
              .then(is => is
                    ? client.click('//button[contains(., "Confirm")]')
                    : [])
              : [])
        .pause(1000)
        .catch(e => console.log(e))
}
module.exports = friendsOnFacebook;


connect add friends facebook?


In [ ]:
var importer = require('../Core');
var glob = require('glob');
var path = require('path');
var fs = require('fs');
var runSeleniumCell = importer.import('run selenium cell');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i])) ;
    return co;
}
                                         
function loadNewConnections() {
    // search posts
    if(fs.existsSync(path.join(project, 'new-friends.json'))) {
        return JSON.parse(fs.readFileSync(path.join(project, 'new-friends.json')).toString());
    }
    var posts = glob.sync('**/*-posts-*', {cwd: project})
        .map(f => path.join(project, f));
    var friends = posts.reduce((acc, p) => {
        importer.streamJson(p, [true, {emitPath: true}], (match) => {
            [].concat(match.value).forEach(m => {
                const url = regexToArray(/facebook.com\/.*/ig, m);
                if(url && url[0]) {
                    acc.push(url[0].replace(/\?fref.*|\?ref.*|\?rc=.*|\?hc_location.*/ig, ''));
                }
            })
        });
        return acc;
    }, []);
    var unique = [];
    for(let i = 0; i < friends.length / 1000; i++) {
        unique = unique.concat(friends.slice(i * 1000, i * 1000 + 1000).filter((u, i, arr) => arr.indexOf(u) === i))
    }
    fs.writeFileSync(path.join(project, 'new-friends.json'), JSON.stringify(unique, null, 4));
    // TODO: diff file-system (abstract better?, code reuse?), selenium connect
    return unique;
}

var friendsOnFacebook, loginFacebook;
function addFacebookFriends() {
    var pages = loadNewConnections();
    console.log(pages.length);
    console.log(pages.slice(0, 30))
    return runSeleniumCell(['log in facebook', 'connect on facebook'])
        .then(r => {
            loginFacebook = r.loginFacebook;
            friendsOnFacebook = r.friendsOnFacebook;
            return loginFacebook();
        })
        .then(() => {
            const rand = Math.round(pages.length / 30 * Math.random()) * 30;
            console.log(pages.slice(rand, rand + 30));
            return importer.runAllPromises(pages.slice(rand, rand + 30).map(page => resolve => {
                console.log(page);
                return friendsOnFacebook(page, true)
                    .then(r => resolve(r))
                    .catch(e => resolve(e))
            }))
        })
}
module.exports = addFacebookFriends;


In [ ]:
$$.async();
addFacebookFriends()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


tell a joke on facebook messenger?

run commands from facebook messenger?

